# arg validators

In [ ]:
if not isinstance(data, pd.core.frame.DataFrame):
    raise TypeError('Expect data as DataFrame')

if not isinstance(data.index,pd.core.indexes.datetimes.DatetimeIndex):
    raise TypeError('Expect data.index as DatetimeIndex')

In [ ]:
same_length = len(opens) == len(highs) == len(lows) == len(closes)
if not same_length:
    raise ValueError('O,H,L,C must have the same length!')

In [ ]:
# MA线
mavprices = pd.Series(prices).rolling(mav).mean().values
ax.plot(xdates, mavprices, linewidth=lw, color=next(mavc))

In [ ]:
# 蜡烛图
def _construct_candlestick_collections(dates, opens, highs, lows, closes, marketcolors=None, config=None):
    """
    Returns
    -------
    ret : list
        (lineCollection, barCollection)
    """

    delta = config['_width_config']['candle_width'] / 2.0
    
    ### 柱四个角的坐标
    barVerts = [((date - delta, open),
                 (date - delta, close),
                 (date + delta, close),
                 (date + delta, open))
                for date, open, close in zip(dates, opens, closes)]

    ### 下影线坐标
    rangeSegLow   = [((date, low), (date, min(open,close)))
                     for date, low, open, close in zip(dates, lows, opens, closes)]
    
    ### 上影线坐标
    rangeSegHigh  = [((date, high), (date, max(open,close)))
                     for date, high, open, close in zip(dates, highs, opens, closes)]
                      
    rangeSegments = rangeSegLow + rangeSegHigh

    alpha  = marketcolors['alpha']

    uc     = mcolors.to_rgba(marketcolors['candle'][ 'up' ], alpha)
    dc     = mcolors.to_rgba(marketcolors['candle']['down'], alpha)
    colors = _updown_colors(uc, dc, opens, closes)

    uc     = mcolors.to_rgba(marketcolors['edge'][ 'up' ], 1.0)
    dc     = mcolors.to_rgba(marketcolors['edge']['down'], 1.0)
    edgecolor = _updown_colors(uc, dc, opens, closes)
    
    uc     = mcolors.to_rgba(marketcolors['wick'][ 'up' ], 1.0)
    dc     = mcolors.to_rgba(marketcolors['wick']['down'], 1.0)
    wickcolor = _updown_colors(uc, dc, opens, closes)

    lw = config['_width_config']['candle_linewidth']

    rangeCollection = LineCollection(rangeSegments,
                                     colors=wickcolor,
                                     linewidths=lw,
                                     )

    barCollection = PolyCollection(barVerts,
                                   facecolors=colors,
                                   edgecolors=edgecolor,
                                   linewidths=lw
                                   )

    return [rangeCollection, barCollection]